# Fashion MNIST

## Validate Azure ML SDK installation and get version number for debugging purposes

In [ ]:
from azureml.core import Experiment, Run, Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace
Initialize a workspace object from persisted configuration.

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep='\n')

In [ ]:
from __future__ import print_function
import os
import numpy as np
from functools import partial
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

## Create a project directory
Create a directory that will contain all the output from this experiment.

In [ ]:
project_folder = '../projects/fashion_mnist_logtoazure'
os.makedirs(project_folder, exist_ok=True)

## Download Fashion MNIST and Prepare Datasets

In [ ]:
import os
import urllib.request

data_folder = '../data/fashion_mnist'
os.makedirs(data_folder, exist_ok = True)
urllib.request.urlretrieve('https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz', filename=os.path.join(data_folder, "train-images.gz"))
urllib.request.urlretrieve('https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, "train-labels.gz"))
urllib.request.urlretrieve('https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz', filename=os.path.join(data_folder, "test-images.gz"))
urllib.request.urlretrieve('https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz', filename=os.path.join(data_folder, "test-labels.gz"))

In [ ]:
import gzip
import struct

def load_data(filename, label=False):
    with gzip.open(filename) as gz:
        struct.unpack('I', gz.read(4))
        n_items = struct.unpack('>I', gz.read(4))
        if not label:
            n_rows = struct.unpack('>I', gz.read(4))[0]
            n_cols = struct.unpack('>I', gz.read(4))[0]
            res = np.frombuffer(gz.read(n_items[0] * n_rows * n_cols), dtype=np.uint8)
            res = res.reshape(n_items[0], n_rows * n_cols)
        else:
            res = np.frombuffer(gz.read(n_items[0]), dtype=np.uint8)
            res = res.reshape(n_items[0], 1)
    return res

In [ ]:
# Number of classes - do not change unless the data changes
num_classes = 10

# sizes of batch and # of epochs of data
batch_size = 128
epochs = 24

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
x_train = load_data(os.path.join(data_folder, 'train-images.gz'), False)
x_test = load_data(os.path.join(data_folder, 'test-images.gz'), False)
y_train = load_data(os.path.join(data_folder, 'train-labels.gz'), True)
y_test = load_data(os.path.join(data_folder, 'test-labels.gz'), True)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

#   Deal with format issues between different backends.  Some put the # of channels in the image before the width and height of image.
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

#   Type convert and scale the test and training data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape (after reshape):', x_train.shape)
print('x_test shape (after reshape):', x_test.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

img_index = 1
plt.imshow(1-x_train[0][:, :, 0], cmap='gray')

In [ ]:
# Label Description 
label_dict = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

In [ ]:
print("Before:\n{}".format(y_train[:4]))
# convert class vectors to binary class matrices.  One-hot encoding
#  3 => 0 0 0 1 0 0 0 0 0 0 and 1 => 0 1 0 0 0 0 0 0 0 0 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("After:\n{}".format(y_train[:4]))  # verify one-hot encoding

## Define the model

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Take a look at the model summary
model.summary()

In [ ]:
#   define compile to minimize categorical loss, use ada delta optimized, and optimize to maximizing accuracy
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

## Setup Experiment

In [ ]:
experiment_name = 'fashion-mnist'
experiment = Experiment(workspace=ws, name=experiment_name)

In [ ]:
experiment

## Train

In [ ]:
#   Define early stopping callback
my_callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, mode='max')]

In [ ]:
with experiment.start_logging(outputs=None, snapshot_directory=project_folder) as run:
    try:
        run.tag("Description","Locally trained Fashion MNIST model")

        #   Train the model and test/validate the mode with the test data after each cycle (epoch) through the training data
        #   Return history of loss and accuracy for each epoch
        hist = model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            callbacks=my_callbacks,
            validation_data=(x_test, y_test))
        run.log_list('Training Accuracy', hist.history['accuracy'])
        run.log_list('Validation Accuracy', hist.history['val_accuracy'])

        #   Evaluate the model with the test data to get the scores on "real" data.
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        run.log('loss', score[0])
        run.log('accuracy', score[1])

        #   Plot data to see relationships in training and validation data
        import numpy as np
        import matplotlib.pyplot as plt
        epoch_list = list(range(1, len(hist.history['accuracy']) + 1))  # values for x axis [1, 2, ..., # of epochs]
        plt.plot(epoch_list, hist.history['accuracy'], epoch_list, hist.history['val_accuracy'])

        plt.legend(('Training Accuracy', 'Validation Accuracy'))
        run.log_image(name='Accuracy', plot=plt)

        run.complete()
    except Exception as e:
        run.fail()
        print(str(e))

In [ ]:
experiment

## Look at some predictions

In [ ]:
def run_prediction(idx):
    result = np.argmax(model.predict(x_test[idx:idx+1]))
    label = np.argmax(y_test[idx])
    print('Prediction: {} ({})'.format(result, label_dict[result]))
    print('Label: {} ({})'.format(label, label_dict[label]))
    #plt.imshow(1-x_test[idx][:, :, 0], cmap='gray')

In [ ]:
import random

for _ in range(1,10):
    idx = random.randint(0, 47-1)
    run_prediction(idx)

## Keras exports

In [ ]:
keras_path = os.path.join(project_folder, "keras")
os.makedirs(keras_path, exist_ok=True)

with open(os.path.join(keras_path, "model.json"), 'w') as f:
    f.write(model.to_json())
model.save_weights(os.path.join(keras_path, 'model.h5'))

model.save(os.path.join(keras_path, 'full_model.h5'))

## Upload model files

In [ ]:
import os
for root, dirs, files in os.walk(project_folder, topdown=False):
    for filename in files:
        source = os.path.join(root, filename)
        name = source.replace(project_folder, "./outputs")
        print("Uploading " + filename)
        run.upload_file(name, source)

In [ ]:
run